# Experiments with the joining validator

In this notebook we experiment with different configurations of the joining validator. In the previous experiments we have been using the default configuration of the joining validator, namely a RandomForestClassifier with 100 trees with a predefined and static threshold. In this notebook we will experiment with different configurations of the joining validator strategy, namely:

- Different classifiers: we will experiment with different classifiers, such as KNN, SVM, and MLP.
- One class classifiers: we will experiment with one class classifiers such as IsolationForest and LocalOutlierFactor.
- Different thresholds: we will experiment with different thresholds to see if privacy/utility trade-offs can be controlled.
- Adaptive vs. static thresholds: we will experiment with adaptive thresholds to see if they can improve the performance of the joining validator.

Alltogether we hope to begin to answer the question; If better or worse performance of the joining validator is desired, what configurations should be used?

For the comparison we will continue to work off the previous experiments, the Hepatitis dataset split into categorical and numerical partitions, and the DataSynthesizer-DPGAN DGM.

In [1]:
### Imports
import pandas as pd

from pandas import DataFrame
from typing import List, Dict

from syntheval import SynthEval

from disjoint_generative_model import DisjointGenerativeModels
from disjoint_generative_model.utils.joining_validator import JoiningValidator, OneClassValidator
from disjoint_generative_model.utils.joining_strategies import UsingJoiningValidator

from experiments.auxiliaries.exp_repeater import repeated_experiment

### Constants
NUM_REPS = 10

### Metrics
metrics = {
    "pca"       : {},
    "h_dist"    : {},
    "corr_diff" : {"mixed_corr": True},
    "auroc_diff" : {"model": "rf_cls"},
    "cls_acc"   : {"F1_type": "macro"},
    "eps_risk"  : {},
    "dcr"       : {},
    "mia"  : {"num_eval_iter": 5},
}

In [2]:
### Load data
df_train = pd.read_csv('experiments/datasets/hepatitis_train.csv')
df_test = pd.read_csv('experiments/datasets/hepatitis_test.csv')

label = 'b_class'

cat_atts = df_train.select_dtypes(include='object').columns.tolist()

for att in [att for att in df_train.columns if att not in cat_atts]:
    if len(df_train[att].unique()) <= 5:
        cat_atts.append(att)

num_atts = [att for att in df_train.columns if att not in cat_atts]

In [3]:
### Define evaluation
SE = SynthEval(df_train, df_test, verbose=False)

### Reload baseline
baseline = pd.read_csv('experiments/results/mixed_model_results/hepatitis_case_study/datasynthesizer_dpgan.csv')
baseline

,Unnamed: 0,mean,sem
0,avg_h_dist,0.069660,0.002684
1,corr_mat_diff,1.320259,0.090476
2,auroc,0.039425,0.006946
3,cls_F1_diff,0.023549,0.004164
4,cls_F1_diff_hout,0.048643,0.008506
5,eps_identif_risk,0.029774,0.007723
6,priv_loss_eps,0.010407,0.003291
7,median_DCR,0.962430,0.027905
8,mia_recall,0.006087,0.001856
9,mia_precision,0.316667,0.072903


## Different classifiers

We will start by experimenting with different classifiers. This experiment aims to explore if the default choice of the RandomForestClassifier is was justified, compared to other obvious choices.

### K-Nearest Neighbours

A K-Nearest Neighbours (KNN) classifier is a simple and interpretable model that is often used for binary classification tasks. Since odd joins will likely fall outside of a hyper-sphere of good joins, a KNN classifier might be a good choice for the joining validator.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [7]:
### Using KNN model with default parameters in joining model with standard behaviour
def knn_base(df_train: DataFrame, id: int = None) -> DataFrame:
    model = KNeighborsClassifier()

    JV = JoiningValidator(classifier_model_base=model)
    JS = UsingJoiningValidator(join_validator_model=JV, min_iter=10, patience=10) # overriding some of the default values to avoid getting too few samples
    dgms = DisjointGenerativeModels(df_train, {'datasynthesizer': cat_atts, 'dpgan': num_atts}, joining_strategy=JS, worker_id=id)
    dgms.join_multiplier = 10

    df_syn = dgms.fit_generate()
    return df_syn

res = repeated_experiment(df_train, df_test, label, knn_base, NUM_REPS, metrics)
res.to_csv('experiments/results/mixed_model_results/validator_case_study/knn_base.csv')

Cross-validated accuracies: [0.5641025641025641, 0.5656108597285068, 0.579185520361991, 0.5776772247360482, 0.5882352941176471]
Mean accuracy: 0.5749622926093514
Final model trained!
Cross-validated accuracies: [0.5806938159879337, 0.5520361990950227, 0.5806938159879337, 0.5686274509803921, 0.5641025641025641]
Mean accuracy: 0.5692307692307692
Final model trained!
Cross-validated accuracies: [0.5897435897435898, 0.5535444947209653, 0.5731523378582202, 0.5912518853695324, 0.5761689291101055]
Mean accuracy: 0.5767722473604826
Cross-validated accuracies: [0.5776772247360482, 0.5746606334841629, 0.5746606334841629, 0.5927601809954751, 0.583710407239819]
Mean accuracy: 0.5806938159879336
Final model trained!
Cross-validated accuracies: [0.5686274509803921, 0.5565610859728507, 0.5490196078431373, 0.6184012066365008, 0.5822021116138764]
Mean accuracy: 0.5749622926093514
Final model trained!
Final model trained!
Cross-validated accuracies: [0.5806938159879337, 0.5897435897435898, 0.58220211161

Exception ignored in: <built-in method release of _thread.lock object at 0x7f1410105cc0>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    pid = os.fork()
RuntimeError: release unlocked lock
Exception ignored in: <built-in method release of _thread.lock object at 0x7fc86871b3c0>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    pid = os.fork()
RuntimeError: release unlocked lock
Exception ignored in: <built-in method release of _thread.lock object at 0x7f78401bf700>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    pid = os.fork()
RuntimeError: release unlocked lock
Exception ignored in: <function release_locks at 0x7f4120471bd0>
Traceback (most r

Adding attribute RNA EOT
Adding attribute Diarrhea
Adding attribute RNA EOTAdding attribute Gender

Adding attribute WBC
Adding attribute Plat
Adding attribute WBC
Adding attribute RNA EOT
Adding attribute Plat
Adding attribute RNA EOT
Adding attribute Plat
Adding attribute RNA 12
Adding attribute WBC
Adding attribute WBC
Adding attribute Nausea/Vomting
Adding attribute RNA 12
Adding attribute Fatigue & generalized bone ache
Adding attribute RNA EF
Adding attribute RNA 12
Adding attribute WBC
Adding attribute Headache
Adding attribute Jaundice
Adding attribute RNA 4
Adding attribute Plat
Adding attribute RNA 4
Adding attribute Diarrhea
Adding attribute RNA 4
Adding attribute Jaundice
Adding attribute Jaundice
Adding attribute RNA 12


[2025-01-13T16:52:28.246906+0100][1597241][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py


Adding attribute RNA 12
Adding attribute Headache


...

 32%|███▏      | 645/2000 [09:07<09:52,  2.29it/s]

Predicted good joins fraction: 0.2553544494720965


 40%|███▉      | 799/2000 [10:22<15:35,  1.28it/s]


Predicted good joins fraction: 0.12684766214177978
Predicted good joins fraction: 0.02988426325790292
Predicted good joins fraction: 0.013710826210826211
Predicted good joins fraction: 0.009207438165733887


ValueError: Cannot take a larger sample than population when 'replace=False'

In [18]:
res

,mean,sem
avg_h_dist,0.036687,0.003163
corr_mat_diff,1.740589,0.144789
auroc,0.038725,0.008903
cls_F1_diff,0.023963,0.004687
cls_F1_diff_hout,0.044789,0.006244
eps_identif_risk,0.030045,0.005549
priv_loss_eps,0.009321,0.002266
median_DCR,1.00582,0.015984
mia_recall,0.004348,0.001163
mia_precision,0.175,0.048512


In [6]:
### Using KNN model with default parameters in joining model with adaptive threshold
def knn_adaptive(df_train: DataFrame, id: int = None) -> DataFrame:
    model = KNeighborsClassifier()

    JV = JoiningValidator(classifier_model_base=model)
    JS = UsingJoiningValidator(join_validator_model=JV, behaviour='adaptive')
    dgms = DisjointGenerativeModels(df_train, {'datasynthesizer': cat_atts, 'dpgan': num_atts}, joining_strategy=JS, worker_id=id)

    df_syn = dgms.fit_generate()
    return df_syn

res = repeated_experiment(df_train, df_test, label, knn_adaptive, NUM_REPS, metrics)
res.to_csv('experiments/results/mixed_model_results/validator_case_study/knn_adaptive.csv')

Cross-validated accuracies: [0.5731523378582202, 0.5686274509803921, 0.5716440422322775, 0.583710407239819, 0.6108597285067874]
Mean accuracy: 0.5815987933634992
Final model trained!
Cross-validated accuracies: [0.5731523378582202, 0.5580693815987934, 0.6018099547511312, 0.5475113122171946, 0.5776772247360482]
Mean accuracy: 0.5716440422322775
Final model trained!
Cross-validated accuracies: [0.5641025641025641, 0.5731523378582202, 0.5716440422322775, 0.5972850678733032, 0.5761689291101055]
Mean accuracy: 0.5764705882352941
Final model trained!
Cross-validated accuracies: [0.5641025641025641, 0.5490196078431373, 0.5972850678733032, 0.55052790346908, 0.5324283559577677]
Mean accuracy: 0.5586726998491705
Final model trained!
Cross-validated accuracies: [0.6048265460030166, 0.5942684766214178, 0.5625942684766214, 0.5746606334841629, 0.5897435897435898]
Mean accuracy: 0.5852187028657616
Cross-validated accuracies: [0.5656108597285068, 0.5610859728506787, 0.5867269984917044, 0.5625942684766

Exception ignored in: <built-in method release of _thread.lock object at 0x7f5928657a00>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    pid = os.fork()
RuntimeError: release unlocked lock
Exception ignored in: <built-in method release of _thread.lock object at 0x7f59c8258680>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    pid = os.fork()
RuntimeError: release unlocked lock
Exception ignored in: <built-in method release of _thread.lock object at 0x7f4d2834fbc0>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/python3.10/site-packages/joblib/externals/loky/backend/fork_exec.py", line 38, in fork_exec
    Exception ignored in: <function release_locks at 0x7f592857db40>
Traceback (most recent call last):
  File "/home/lautrup/sdg_env/lib/

Adding attribute Diarrhea
Adding attribute WBC
Adding attribute Gender
Adding attribute RNA EOT
Adding attribute RNA EOT
Adding attribute Plat
Adding attribute WBC
Adding attribute RNA EOT
Adding attribute Plat
Adding attribute RNA EOT
Adding attribute RNA 12
Adding attribute WBC
Adding attribute Plat
Adding attribute Fatigue & generalized bone ache
Adding attribute WBC
Adding attribute RNA EF
Adding attribute WBC
Adding attribute Nausea/Vomting
Adding attribute RNA 12
Adding attribute RNA 12
Adding attribute RNA 4
Adding attribute Jaundice
Adding attribute Headache
Adding attribute Jaundice
Adding attribute Plat
Adding attribute Diarrhea
Adding attribute RNA 4
Adding attribute RNA 12
Adding attribute Jaundice
Adding attribute RNA 4


[2025-01-13T16:34:16.266146+0100][1551021][CRITICAL] module disabled: /home/lautrup/sdg_env/lib/python3.10/site-packages/synthcity/plugins/generic/plugin_goggle.py


Adding attribute Headache
Adding attribute Gender
Adding attribute RNA 12
Adding attribute Plat
Adding attribute Jaundice
Adding attribute Gender
Adding attribute Headache
Adding attribute Headache
Adding attribute Gender
Adding attribute Gender


...

 35%|███▍      | 699/2000 [08:59<16:43,  1.30it/s]


Threshold auto-set to: 0.6
Predicted good joins fraction: 0.3704374057315234


 37%|███▋      | 749/2000 [09:20<15:35,  1.34it/s]


Threshold auto-set to: 0.4
Predicted good joins fraction: 0.38552036199095024


/home/lautrup/sdg_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lautrup/sdg_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lautrup/sdg_env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/lautrup/sdg_env/lib/python3.10/sit

In [6]:
res

,mean,sem
avg_h_dist,0.033838,0.001977
corr_mat_diff,1.385155,0.104282
auroc,0.030347,0.008432
cls_F1_diff,0.032157,0.004103
cls_F1_diff_hout,0.045835,0.006292
eps_identif_risk,0.030588,0.00632
priv_loss_eps,0.008235,0.001876
median_DCR,0.983746,0.014069
mia_recall,0.003188,0.000676
mia_precision,0.153333,0.046401
